# Clasificador de Desechos 🗑️

🧑🏽‍💻 **Said Ortigoza Trujillo, A01707430**

### Conexión con Google Drive y visualización del directorio

In [66]:
from google.colab import drive
drive.mount('/content/drive')

# Path en Drive de la carpeta del dataset
%cd "/content/drive/MyDrive/TEC/Semestre 8/Waste Classification"
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TEC/Semestre 8/Waste Classification
Dataset  Test  Train  Validation


### Importar las librerías necesarias

**tensorflow (tf):** Proporciona herramientas para construir, entrenar y desplegar modelos de redes neuronales.

**pandas (pd):** Ofrece estructuras de datos y funciones para trabajar con datos tabulares y series temporales de manera eficiente.

**numpy (np):** Proporciona soporte para matrices multidimensionales y una amplia gama de funciones matemáticas para operar con estas matrices de manera eficiente.

**matplotlib.pyplot (plt):** Para crear gráficos, diagramas y trazados para explorar y comunicar datos.

**os:** Se utiliza para manipular archivos, directorios y rutas de archivos.

**shutil:** Se utiliza para copiar, mover y eliminar archivos y directorios.

**random:** Se utiliza para generar secuencias de números aleatorios y para mezclar aleatoriamente elementos de una lista.

**tensorflow.keras.preprocessing.image.ImageDataGenerator:** Se utiliza para cargar y preprocesar imágenes para su uso en modelos de redes neuronales.

**glob:** Se utiliza para buscar y listar archivos en un directorio.

**PIL.Image:** Se utiliza para abrir, manipular y guardar imágenes en varios formatos de archivo.

In [67]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob
from PIL import Image

### Directorios del dataset inicial, entrenamiento, prueba y validación

In [13]:
# Rutas de las carpetas
base_dir = 'Dataset'
train_dir = 'Train'
test_dir = 'Test'
val_dir = 'Validation'

### Creación de los subdirectorios de cada conjunto, a partir del directorio base

In [14]:
classes = [os.path.basename(d) for d in glob(os.path.join(base_dir, '*')) if os.path.isdir(d)]
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

### Configuración de las proporciones del split

In [15]:
train_split = 0.8
test_split = 0.20
val_split = 0.10

### División y reubicación de los archivos del directorio base en entrenamiento, prueba y validación

In [16]:
for cls in classes:
    cls_path = os.path.join(base_dir, cls)
    images = glob(os.path.join(cls_path, '*'))

    # Mezclar las imágenes
    random.shuffle(images)

    # Calcular los índices para división
    train_size = int(len(images) * train_split)
    val_size = int(len(images) * val_split)
    test_size = len(images) - train_size - val_size

    # Dividir las imágenes
    train_images = images[:train_size]
    val_images = images[train_size:train_size+val_size]
    test_images = images[train_size+val_size:]

    # Mover las imágenes a las carpetas correspondientes
    for img in train_images:
        shutil.move(img, os.path.join(train_dir, cls))
    for img in val_images:
        shutil.move(img, os.path.join(val_dir, cls))
    for img in test_images:
        shutil.move(img, os.path.join(test_dir, cls))

### Configuración del generador de datos

In [68]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.05,
    horizontal_flip = True,
    fill_mode='reflect'
)

val_test_datagen = ImageDataGenerator(rescale = 1/255)

### Cargar los tres conjuntos

In [89]:
# Cargar el conjunto de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (500, 500),
    batch_size = 5,
    class_mode ='binary'
)

# Cargar el conjunto de validación
validation_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size = (500, 500),
    batch_size = 5,
    class_mode ='binary'
)

# Cargar el conjunto de prueba
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size = (500, 500),
    batch_size = 5,
    class_mode ='binary'
)

Found 3798 images belonging to 9 classes.
Found 473 images belonging to 9 classes.
Found 481 images belonging to 9 classes.


### Visualización de imágenes generadas con data augmentation

In [90]:
plt.figure()
#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(1, 5, figsize=(30, 8))

for i in range(5):
	# Hasta aquí empieza a generar las imágenes
  axarr[i].imshow(train_generator[0][0][0])

<Figure size 640x480 with 0 Axes>